In [3]:
import torch
import time

# --- 1. GPU 설정 확인 ---
if torch.cuda.is_available():
    # 사용 가능한 GPU가 있는지 확인
    print("✅ GPU를 사용할 수 있습니다.")
    
    # 현재 사용 중인 GPU 장치 번호 확인
    gpu_id = torch.cuda.current_device()
    print(f"   - GPU 장치: {torch.cuda.get_device_name(gpu_id)}")
    
    # 사용할 장치를 'cuda'로 설정
    device = torch.device("cuda")
    
else:
    print("❌ GPU를 사용할 수 없습니다. CPU를 사용합니다.")
    device = torch.device("cpu")

print("="*50)


# --- 2. 속도 비교를 위한 대규모 텐서(행렬) 생성 ---
# 비교할 텐서의 크기 (숫자를 키울수록 차이가 극명해집니다)
tensor_size = 20000

# CPU에 텐서 생성
cpu_tensor_a = torch.randn(tensor_size, tensor_size, device='cpu')
cpu_tensor_b = torch.randn(tensor_size, tensor_size, device='cpu')

# GPU에 동일한 텐서 생성 (GPU 사용 가능 시)
if device.type == 'cuda':
    gpu_tensor_a = cpu_tensor_a.to(device)
    gpu_tensor_b = cpu_tensor_b.to(device)


# --- 3. CPU 연산 속도 측정 ---
print(f"[{tensor_size}x{tensor_size}] 행렬 곱셈 속도를 비교합니다...")

start_time_cpu = time.time()
result_cpu = torch.matmul(cpu_tensor_a, cpu_tensor_b)
end_time_cpu = time.time()

cpu_duration = end_time_cpu - start_time_cpu
print(f"   - 💻 CPU 연산 시간: {cpu_duration:.4f} 초")


# --- 4. GPU 연산 속도 측정 ---
if device.type == 'cuda':
    # GPU 워밍업 (첫 연산은 초기화 때문에 느릴 수 있어 한번 실행해 줌)
    _ = torch.matmul(gpu_tensor_a, gpu_tensor_b)
    
    start_time_gpu = time.time()
    result_gpu = torch.matmul(gpu_tensor_a, gpu_tensor_b)
    #torch.cuda.synchronize() # ✅ 이 줄을 추가하여 CPU가 GPU 연산을 기다리게 함
    end_time_gpu = time.time()

    gpu_duration = end_time_gpu - start_time_gpu
    print(f"   - 🚀 GPU 연산 시간: {gpu_duration:.4f} 초")
    
    print("="*50)
    print(f"🔥 속도 차이: GPU가 CPU보다 약 {cpu_duration / gpu_duration:.1f} 배 빠릅니다.")

# 더 이상 사용하지 않는 GPU 텐서 변수 삭제
del gpu_tensor_a
del gpu_tensor_b
del result_gpu

# PyTorch가 VRAM 캐시를 비우도록 강제
torch.cuda.empty_cache()

✅ GPU를 사용할 수 있습니다.
   - GPU 장치: NVIDIA GeForce RTX 5070
[20000x20000] 행렬 곱셈 속도를 비교합니다...
   - 💻 CPU 연산 시간: 8.2219 초
   - 🚀 GPU 연산 시간: 0.0000 초
🔥 속도 차이: GPU가 CPU보다 약 201667.7 배 빠릅니다.
